In [1]:
import pyspark
from pyspark.sql import SparkSession
import findspark# solved Python worker failed to connect back
findspark.init()# solved Python worker failed to connect back

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
from datetime import datetime

In [5]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [6]:
rows = rdd.take(10)
row = rows[0]

In [7]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 3, 19, 0, 1), PULocationID=244, total_amount=8.8)

In [8]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [9]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [10]:
from collections import namedtuple

In [11]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [12]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [13]:
from pyspark.sql import types

In [14]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [15]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [16]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [17]:
import pandas as pd

In [18]:
rows = duration_rdd.take(10)

In [19]:
df = pd.DataFrame(rows, columns=columns)

In [20]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [ ]:
#Try basic prediction

In [21]:
def model_predict(df):
    y_pred = df.trip_distance * 5
    return y_pred

In [22]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [23]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [24]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|               5.0|
|               4.1|
|               0.0|
|              24.4|
|1.7000000000000002|
|              7.95|
|               3.7|
|              21.7|
|28.700000000000003|
|              32.5|
|               4.9|
|             13.25|
|              13.4|
|               0.0|
|             52.75|
|             26.25|
|             31.75|
|              17.5|
|              10.2|
|              14.0|
+------------------+
only showing top 20 rows

